In [ ]:
# import torch
# torch.cuda.empty_cache()

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Fake_news')

import os
from google.colab import drive
MOUNTPOINT = '/content/drive'
DATADIR = os.path.join(MOUNTPOINT, 'MyDrive', 'Fake_news')
drive.mount(MOUNTPOINT)

Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 14.0 MB/s 
     |████████████████████████████████| 101 kB 11.6 MB/s 
     |████████████████████████████████| 6.6 MB 57.8 MB/s 
     |████████████████████████████████| 596 kB 75.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import numpy as np
import json, re
from tqdm import tqdm_notebook
from uuid import uuid4

## Torch Modules
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

In [ ]:
# Use GPU
device = torch.device("cuda")

In [ ]:
from imblearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
from sklearn.utils import shuffle

In [ ]:
df_train = pd.read_csv("Constraint_Train_p.csv")
df_test = pd.read_csv("Constraint_Test_p.csv")
df_val = pd.read_csv("Constraint_Val_p.csv")

In [ ]:
df = pd.concat([df_train,df_val],axis=0,ignore_index=True)

In [ ]:
df = shuffle(df, random_state=0)
sentences = df['text'] 
labels = df['label']

In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 11.8 MB/s 


In [ ]:
from transformers import (BertForSequenceClassification,BertTokenizer,
                          RobertaForSequenceClassification,RobertaTokenizer,
                         AdamW)
from transformers import XLNetTokenizer, XLNetModel, XLNetForSequenceClassification
import torch

from transformers import (
  AutoModel,
   AutoConfig,
   AutoTokenizer,
   TFAutoModelForSequenceClassification,
   AdamW,
   glue_convert_examples_to_features,
   BertConfig,
   AutoModelForSequenceClassification
)
import tensorflow as tf
import tensorflow_datasets as tfds
import json

# BERT
bert_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
                                                                num_labels = 2, # The number of output labels--2 for binary classification.
                                                                                # You can increase this for multi-class tasks.   
                                                                output_attentions = False, # Whether the model returns attentions weights.
                                                                output_hidden_states = False # Whether the model returns all hidden-states.
                                                          )
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model.cuda()
                                                           

# RoBERTa
roberta_model = RobertaForSequenceClassification.from_pretrained("roberta-base", # 12-layer, 768-hidden, 12-heads, 125M parameters RoBERTa using the BERT-base architecture
                                                                    num_labels = 2, # The number of output labels--2 for binary classification.
                                                                                    # You can increase this for multi-class tasks.   
                                                                    output_attentions = False, # Whether the model returns attentions weights.
                                                                    output_hidden_states = False # Whether the model returns all hidden-states.
                                                                )
roberta_tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
roberta_model.cuda()

# XLNet
xlnet_model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels = 2, output_attentions = False, output_hidden_states = False)
xlnet_tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")

xlnet_model.cuda()

# CT-BERT
# ctbert_model = AutoModel.from_pretrained("digitalepidemiologylab/covid-twitter-bert", num_labels = 2, output_attentions = False, output_hidden_states = False)
ctbert_tokenizer = AutoTokenizer.from_pretrained("digitalepidemiologylab/covid-twitter-bert")

config = BertConfig.from_pretrained("digitalepidemiologylab/covid-twitter-bert", num_labels = 2, output_attentions = False, output_hidden_states = False)    # Download configuration from S3 and cache.
ctbert_model = AutoModelForSequenceClassification.from_config(config) 
ctbert_model.cuda()

# BT-BERT
btbert_tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base")

bt_config = BertConfig.from_pretrained("vinai/bertweet-base", num_labels = 2, output_attentions = False, output_hidden_states = False)    # Download configuration from S3 and cache.
btbert_model = AutoModelForSequenceClassification.from_config(bt_config) 

btbert_model.cuda()

In [ ]:
max_len_bert = 0
max_len_roberta = 0
max_len_xlnet = 0
max_len_ctbert = 0
max_len_btbert = 0



# For every sentence...
for sent in sentences:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids_bert = bert_tokenizer.encode(sent, add_special_tokens=True)
    input_ids_roberta = roberta_tokenizer.encode(sent, add_special_tokens=True)
    input_ids_xlnet = xlnet_tokenizer.encode(sent, add_special_tokens=True)
    input_ids_ctbert = ctbert_tokenizer.encode(sent, add_special_tokens=True)
    input_ids_btbert = btbert_tokenizer.encode(sent, add_special_tokens=True)


    # Update the maximum sentence length.
    max_len_bert = max(max_len_bert, len(input_ids_bert))
    max_len_roberta = max(max_len_roberta, len(input_ids_roberta))
    max_len_xlnet = max(max_len_xlnet, len(input_ids_xlnet))
    max_len_ctbert = max(max_len_ctbert, len(input_ids_ctbert))
    max_len_btbert = max(max_len_btbert, len(input_ids_btbert))

Token indices sequence length is longer than the specified maximum sequence length for this model (368 > 128). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1094 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1126 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
bert_input_ids = []
bert_attention_masks = []
roberta_input_ids = []
roberta_attention_masks = []
xlnet_input_ids = []
xlnet_attention_masks = []
ctbert_input_ids = []
ctbert_attention_masks = []
btbert_input_ids = []
btbert_attention_masks = []
sentence_ids = []
counter = 0

# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    bert_encoded_dict = bert_tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 120,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    
    roberta_encoded_dict = roberta_tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 120,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    xlnet_encoded_dict = xlnet_tokenizer.encode_plus(
                    sent,                      # Sentence to encode.
                    add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                    max_length = 120,           # Pad & truncate all sentences.
                    pad_to_max_length = True,
                    return_attention_mask = True,   # Construct attn. masks.
                    return_tensors = 'pt',     # Return pytorch tensors.
                )
    
    ctbert_encoded_dict = ctbert_tokenizer.encode_plus(
                    sent,                      # Sentence to encode.
                    add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                    max_length = 120,           # Pad & truncate all sentences.
                    pad_to_max_length = True,
                    return_attention_mask = True,   # Construct attn. masks.
                    return_tensors = 'pt',     # Return pytorch tensors.
                )
    
    btbert_encoded_dict = btbert_tokenizer.encode_plus(
                sent,                      # Sentence to encode.
                add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                max_length = 120,           # Pad & truncate all sentences.
                pad_to_max_length = True,
                return_attention_mask = True,   # Construct attn. masks.
                return_tensors = 'pt',     # Return pytorch tensors.
            )

    # Add the encoded sentence to the list.    
    bert_input_ids.append(bert_encoded_dict['input_ids'])
    roberta_input_ids.append(roberta_encoded_dict['input_ids'])
    xlnet_input_ids.append(xlnet_encoded_dict['input_ids'])
    ctbert_input_ids.append(ctbert_encoded_dict['input_ids'])
    btbert_input_ids.append(btbert_encoded_dict['input_ids'])

    
    # And its attention mask (simply differentiates padding from non-padding).
    bert_attention_masks.append(bert_encoded_dict['attention_mask'])
    roberta_attention_masks.append(roberta_encoded_dict['attention_mask'])
    xlnet_attention_masks.append(xlnet_encoded_dict['attention_mask'])
    ctbert_attention_masks.append(ctbert_encoded_dict['attention_mask'])
    btbert_attention_masks.append(btbert_encoded_dict['attention_mask'])
    
    
    # collecting sentence_ids
    sentence_ids.append(counter)
    counter  = counter + 1
    
# Convert the lists into tensors.
bert_input_ids = torch.cat(bert_input_ids, dim=0)
bert_attention_masks = torch.cat(bert_attention_masks, dim=0)

roberta_input_ids = torch.cat(roberta_input_ids, dim=0)
roberta_attention_masks = torch.cat(roberta_attention_masks, dim=0)

xlnet_input_ids = torch.cat(xlnet_input_ids, dim=0)
xlnet_attention_masks = torch.cat(xlnet_attention_masks, dim=0)

ctbert_input_ids = torch.cat(ctbert_input_ids, dim=0)
ctbert_attention_masks = torch.cat(ctbert_attention_masks, dim=0)


btbert_input_ids = torch.cat(btbert_input_ids, dim=0)
btbert_attention_masks = torch.cat(btbert_attention_masks, dim=0)

labels = torch.tensor(labels)
sentence_ids = torch.tensor(sentence_ids)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `t

In [ ]:
def index_remover(tensordata):
    input_ids = []
    attention_masks = []
    labels = []
   
    for a,b,c,d in tensordata:
        input_ids.append(b.tolist())
        attention_masks.append(c.tolist())
        labels.append(d.tolist())
        
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)
    labels = torch.tensor(labels)
    
    final_dataset =  TensorDataset(input_ids, attention_masks, labels)
    return final_dataset

In [ ]:
from torch.utils.data import TensorDataset, random_split
# function to seed the script globally
torch.manual_seed(0)

# Combine the training inputs into a TensorDataset.
bert_dataset = TensorDataset(sentence_ids, bert_input_ids, bert_attention_masks, labels)
roberta_dataset = TensorDataset(roberta_input_ids, roberta_attention_masks, labels)
xlnet_dataset = TensorDataset(xlnet_input_ids, xlnet_attention_masks, labels)
ctbert_dataset = TensorDataset(sentence_ids, ctbert_input_ids, ctbert_attention_masks, labels)
btbert_dataset = TensorDataset(sentence_ids, btbert_input_ids, btbert_attention_masks, labels)





In [ ]:
train_size = int(0.8 * len(roberta_dataset))
val_size = len(roberta_dataset) - train_size

# Divide the dataset by randomly selecting samples.
# bert_train_dataset, bert_val_dataset = random_split(bert_dataset, [train_size, val_size])
# roberta_train_dataset, roberta_val_dataset = random_split(roberta_dataset, [train_size, val_size])
# xlnet_train_dataset, xlnet_val_dataset = random_split(xlnet_dataset, [train_size, val_size])

bert_train_dataset = torch.utils.data.Subset(bert_dataset, range(train_size))
bert_val_dataset = torch.utils.data.Subset(bert_dataset, range(train_size, train_size + val_size))
roberta_train_dataset = torch.utils.data.Subset(roberta_dataset, range(train_size))
roberta_val_dataset = torch.utils.data.Subset(roberta_dataset, range(train_size, train_size + val_size))
xlnet_train_dataset = torch.utils.data.Subset(xlnet_dataset, range(train_size))
xlnet_val_dataset = torch.utils.data.Subset(xlnet_dataset, range(train_size, train_size + val_size))
ctbert_train_dataset = torch.utils.data.Subset(ctbert_dataset, range(train_size))
ctbert_val_dataset = torch.utils.data.Subset(ctbert_dataset, range(train_size, train_size + val_size))
btbert_train_dataset = torch.utils.data.Subset(btbert_dataset, range(train_size))
btbert_val_dataset = torch.utils.data.Subset(btbert_dataset, range(train_size, train_size + val_size))

bert_train_dataset = index_remover(bert_train_dataset)
bert_val_dataset = index_remover(bert_val_dataset)
ctbert_train_dataset = index_remover(ctbert_train_dataset)
ctbert_val_dataset = index_remover(ctbert_val_dataset)
btbert_train_dataset = index_remover(btbert_train_dataset)
btbert_val_dataset = index_remover(btbert_val_dataset)

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
bert_train_dataloader = DataLoader(
            bert_train_dataset,  # The training samples.
            sampler = RandomSampler(bert_train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )
roberta_train_dataloader = DataLoader(
            roberta_train_dataset,  # The training samples.
            sampler = RandomSampler(roberta_train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

xlnet_train_dataloader = DataLoader(
            xlnet_train_dataset,  # The training samples.
            sampler = RandomSampler(xlnet_train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

ctbert_train_dataloader = DataLoader(
            ctbert_train_dataset,  # The training samples.
            sampler = RandomSampler(ctbert_train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )


btbert_train_dataloader = DataLoader(
            btbert_train_dataset,  # The training samples.
            sampler = RandomSampler(btbert_train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )
# For validation the order doesn't matter, so we'll just read them sequentially.
bert_validation_dataloader = DataLoader(
            bert_val_dataset, # The validation samples.
            sampler = SequentialSampler(bert_val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

roberta_validation_dataloader = DataLoader(
            roberta_val_dataset, # The validation samples.
            sampler = SequentialSampler(roberta_val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )
xlnet_validation_dataloader = DataLoader(
            xlnet_val_dataset, # The validation samples.
            sampler = SequentialSampler(xlnet_val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )
ctbert_validation_dataloader = DataLoader(
            ctbert_val_dataset, # The validation samples.
            sampler = SequentialSampler(ctbert_val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )
btbert_validation_dataloader = DataLoader(
            btbert_val_dataset, # The validation samples.
            sampler = SequentialSampler(btbert_val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [ ]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
bert_optimizer = AdamW(bert_model.parameters(),
                  lr = 5e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

roberta_optimizer = AdamW(roberta_model.parameters(),
                  lr = 5e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
xlnet_optimizer = AdamW(xlnet_model.parameters(),
                  lr = 5e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

ctbert_optimizer = AdamW(ctbert_model.parameters(),
                  lr = 5e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
btbert_optimizer = AdamW(btbert_model.parameters(),
                  lr = 5e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 2,I have already seen that the model starts overfitting beyound 2 epochs
epochs = 30

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(bert_train_dataloader) * epochs

# Create the learning rate scheduler.
bert_scheduler = get_linear_schedule_with_warmup(bert_optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

roberta_scheduler = get_linear_schedule_with_warmup(roberta_optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)
xlnet_scheduler = get_linear_schedule_with_warmup(xlnet_optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

ctbert_scheduler = get_linear_schedule_with_warmup(ctbert_optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)
btbert_scheduler = get_linear_schedule_with_warmup(btbert_optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)


In [ ]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
import random
seed_val = 100

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
btbert_training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

for epoch_i in range(0, epochs):
    btbert_outputs_prob = []
  
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0
    btbert_model.train()

    for step, batch in enumerate(btbert_train_dataloader):
        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
      
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(btbert_train_dataloader), elapsed))
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        btbert_model.zero_grad()

        outputs = btbert_model(b_input_ids, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.

        total_train_loss += outputs.loss.item()

        # Perform a backward pass to calculate the gradients.
        outputs.loss.backward()
        torch.nn.utils.clip_grad_norm_(btbert_model.parameters(), 1.0)

        btbert_optimizer.step()
        btbert_scheduler.step()

    avg_train_loss = total_train_loss / len(btbert_train_dataloader)            
    
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    btbert_model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in btbert_validation_dataloader:

        
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        with torch.no_grad():
       

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # Get the "logits" output by the roberta_model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            outputs = btbert_model(b_input_ids, 
#                                    token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
        # Accumulate the validation loss.
        total_eval_loss += outputs.loss.item()

        # Move logits and labels to CPU
        outputs.logits = outputs.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        for logit in outputs.logits:
          btbert_outputs_prob.append(logit.tolist())

        total_eval_accuracy += flat_accuracy(outputs.logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(btbert_validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(btbert_validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    btbert_training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    214.    Elapsed: 0:00:15.
  Batch    80  of    214.    Elapsed: 0:00:29.
  Batch   120  of    214.    Elapsed: 0:00:44.
  Batch   160  of    214.    Elapsed: 0:00:58.
  Batch   200  of    214.    Elapsed: 0:01:13.

  Average training loss: 0.01
  Training epcoh took: 0:01:18

Running Validation...
  Accuracy: 0.95
  Validation Loss: 0.31
  Validation took: 0:00:06

======== Epoch 2 / 4 ========
Training...
  Batch    40  of    214.    Elapsed: 0:00:15.
  Batch    80  of    214.    Elapsed: 0:00:29.
  Batch   120  of    214.    Elapsed: 0:00:44.
  Batch   160  of    214.    Elapsed: 0:00:58.
  Batch   200  of    214.    Elapsed: 0:01:13.

  Average training loss: 0.01
  Training epcoh took: 0:01:18

Running Validation...
  Accuracy: 0.95
  Validation Loss: 0.31
  Validation took: 0:00:06

======== Epoch 3 / 4 ========
Training...
  Batch    40  of    214.    Elapsed: 0:00:15.
  Batch    80  of    214.    Elapsed: 0:00:29.
 

In [ ]:
df_bt_stats = pd.DataFrame(data=btbert_training_stats)
df_bt_stats = df_bt_stats.set_index('epoch')
df_bt_stats.to_csv('df_bt_stats_4.csv') 

In [ ]:
df_bt_stats = pd.DataFrame(data=btbert_training_stats)
df_bt_stats = df_bt_stats.set_index('epoch')
df_bt_stats.to_csv('df_bt_stats.csv')  
np_btbert_outputs_prob = np.array([np.array(xi) for xi in btbert_outputs_prob])
np.save('btbert_outputs_prob.npy',np_btbert_outputs_prob)

In [ ]:
df_bt_stats.to_csv('df_bt_stats.csv')  

In [ ]:
np_btbert_outputs_prob = np.array([np.array(xi) for xi in btbert_outputs_prob])
np.save('btbert_outputs_prob.npy',np_btbert_outputs_prob)

In [ ]:
np.save('btbert_outputs_prob.npy',np_btbert_outputs_prob)

array([[-2.87617421,  2.11003757],
       [-3.77582765,  3.07525754],
       [-3.50905228,  2.76657724],
       ...,
       [-3.73377061,  3.02959466],
       [ 3.89040422, -3.43641305],
       [ 3.89586043, -3.44005728]])

In [ ]:
import random
seed_val = 100

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
ctbert_training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

for epoch_i in range(0, epochs):
    ctbert_outputs_prob = []
  
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0
    ctbert_model.train()

    for step, batch in enumerate(ctbert_train_dataloader):
        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
      
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(ctbert_train_dataloader), elapsed))
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        ctbert_model.zero_grad()

        outputs = ctbert_model(b_input_ids, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.

        total_train_loss += outputs.loss.item()

        # Perform a backward pass to calculate the gradients.
        outputs.loss.backward()
        torch.nn.utils.clip_grad_norm_(ctbert_model.parameters(), 1.0)

        ctbert_optimizer.step()
        ctbert_scheduler.step()

    avg_train_loss = total_train_loss / len(ctbert_train_dataloader)            
    
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    ctbert_model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in ctbert_validation_dataloader:

        
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        with torch.no_grad():
       

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # Get the "logits" output by the roberta_model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            outputs = ctbert_model(b_input_ids, 
#                                    token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
        # Accumulate the validation loss.
        total_eval_loss += outputs.loss.item()

        # Move logits and labels to CPU
        outputs.logits = outputs.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        for logit in outputs.logits:
          ctbert_outputs_prob.append(logit.tolist())

        total_eval_accuracy += flat_accuracy(outputs.logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(ctbert_validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(ctbert_validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    ctbert_training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 5 ========
Training...
  Batch    40  of    214.    Elapsed: 0:00:48.
  Batch    80  of    214.    Elapsed: 0:01:35.
  Batch   120  of    214.    Elapsed: 0:02:21.
  Batch   160  of    214.    Elapsed: 0:03:07.
  Batch   200  of    214.    Elapsed: 0:03:53.

  Average training loss: 0.73
  Training epcoh took: 0:04:09

Running Validation...
  Accuracy: 0.52
  Validation Loss: 0.69
  Validation took: 0:00:20

======== Epoch 2 / 5 ========
Training...
  Batch    40  of    214.    Elapsed: 0:00:46.
  Batch    80  of    214.    Elapsed: 0:01:32.
  Batch   120  of    214.    Elapsed: 0:02:18.
  Batch   160  of    214.    Elapsed: 0:03:05.
  Batch   200  of    214.    Elapsed: 0:03:51.

  Average training loss: 0.71
  Training epcoh took: 0:04:07

Running Validation...
  Accuracy: 0.48
  Validation Loss: 0.70
  Validation took: 0:00:20

======== Epoch 3 / 5 ========
Training...
  Batch    40  of    214.    Elapsed: 0:00:46.
  Batch    80  of    214.    Elapsed: 0:01:32.
 

In [ ]:
import random
seed_val = 100

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
bert_training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

for epoch_i in range(0, epochs):
    bert_outputs_prob = []
  
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0
    bert_model.train()

    for step, batch in enumerate(bert_train_dataloader):
        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
      
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(bert_train_dataloader), elapsed))
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        bert_model.zero_grad()        

        outputs = bert_model(b_input_ids, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.

        total_train_loss += outputs.loss.item()

        # Perform a backward pass to calculate the gradients.
        outputs.loss.backward()
        torch.nn.utils.clip_grad_norm_(bert_model.parameters(), 1.0)

        bert_optimizer.step()
        bert_scheduler.step()

    avg_train_loss = total_train_loss / len(bert_train_dataloader)            
    
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    bert_model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in bert_validation_dataloader:
        
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        with torch.no_grad():
       

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # Get the "logits" output by the roberta_model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            outputs = bert_model(b_input_ids, 
#                                    token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
        # Accumulate the validation loss.
        total_eval_loss += outputs.loss.item()

        # Move logits and labels to CPU
        outputs.logits = outputs.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        for logit in outputs.logits:
          bert_outputs_prob.append(logit.tolist())

        total_eval_accuracy += flat_accuracy(outputs.logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(bert_validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(bert_validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    bert_training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 5 ========
Training...
  Batch    40  of    214.    Elapsed: 0:00:15.
  Batch    80  of    214.    Elapsed: 0:00:30.
  Batch   120  of    214.    Elapsed: 0:00:45.
  Batch   160  of    214.    Elapsed: 0:01:00.
  Batch   200  of    214.    Elapsed: 0:01:14.

  Average training loss: 0.21
  Training epcoh took: 0:01:20

Running Validation...
  Accuracy: 0.97
  Validation Loss: 0.10
  Validation took: 0:00:06

======== Epoch 2 / 5 ========
Training...
  Batch    40  of    214.    Elapsed: 0:00:15.
  Batch    80  of    214.    Elapsed: 0:00:30.
  Batch   120  of    214.    Elapsed: 0:00:45.
  Batch   160  of    214.    Elapsed: 0:00:59.
  Batch   200  of    214.    Elapsed: 0:01:14.

  Average training loss: 0.07
  Training epcoh took: 0:01:19

Running Validation...
  Accuracy: 0.97
  Validation Loss: 0.08
  Validation took: 0:00:06

======== Epoch 3 / 5 ========
Training...
  Batch    40  of    214.    Elapsed: 0:00:15.
  Batch    80  of    214.    Elapsed: 0:00:30.
 

In [ ]:
df_b_stats = pd.DataFrame(data=bert_training_stats)
df_b_stats = df_b_stats.set_index('epoch')
df_b_stats.to_csv('df_b_stats.csv')  
np_bert_outputs_prob = np.array([np.array(xi) for xi in bert_outputs_prob])
np.save('bert_outputs_prob.npy',np_bert_outputs_prob)

In [ ]:
import random

# This training code is based on the `run_glue.py` script here:
# a

# Set the seed value all over the place to make this reproducible.
seed_val = 100

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
roberta_training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    roberta_outputs_prob = []
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the roberta_model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-roberta_model-train-do-in-pytorch)
    roberta_model.train()

    # For each batch of training data...
    for step, batch in enumerate(roberta_train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(roberta_train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)


        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        roberta_model.zero_grad()        

        # Perform a forward pass (evaluate the roberta_model on this training batch).
        # The documentation for this `roberta_model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/roberta_model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # are given and what flags are set. For our usage here, it returns
        # the loss (because we provided labels) and the "logits"--the roberta_model
        # outputs prior to activation.


        outputs = roberta_model(b_input_ids, 
                            #  token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.

        total_train_loss += outputs.loss.item()

        # Perform a backward pass to calculate the gradients.
        outputs.loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(roberta_model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The roberta_optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        roberta_optimizer.step()

        # Update the learning rate.
        roberta_scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(roberta_train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the roberta_model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    roberta_model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in roberta_validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():
       

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # Get the "logits" output by the roberta_model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            outputs = roberta_model(b_input_ids, 
#                                    token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
        # Accumulate the validation loss.
        total_eval_loss += outputs.loss.item()

        for logit in outputs.logits:
          roberta_outputs_prob.append(logit.tolist())

        # Move logits and labels to CPU
        outputs.logits = outputs.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(outputs.logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(roberta_validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(roberta_validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    roberta_training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 5 ========
Training...
  Batch    40  of    214.    Elapsed: 0:00:17.
  Batch    80  of    214.    Elapsed: 0:00:35.
  Batch   120  of    214.    Elapsed: 0:00:52.
  Batch   160  of    214.    Elapsed: 0:01:09.
  Batch   200  of    214.    Elapsed: 0:01:24.

  Average training loss: 0.24
  Training epcoh took: 0:01:30

Running Validation...
  Accuracy: 0.97
  Validation Loss: 0.08
  Validation took: 0:00:06

======== Epoch 2 / 5 ========
Training...
  Batch    40  of    214.    Elapsed: 0:00:15.
  Batch    80  of    214.    Elapsed: 0:00:30.
  Batch   120  of    214.    Elapsed: 0:00:45.
  Batch   160  of    214.    Elapsed: 0:01:00.
  Batch   200  of    214.    Elapsed: 0:01:14.

  Average training loss: 0.11
  Training epcoh took: 0:01:20

Running Validation...
  Accuracy: 0.98
  Validation Loss: 0.07
  Validation took: 0:00:06

======== Epoch 3 / 5 ========
Training...
  Batch    40  of    214.    Elapsed: 0:00:15.
  Batch    80  of    214.    Elapsed: 0:00:30.
 

In [ ]:
df_r_stats = pd.DataFrame(data=roberta_training_stats)
df_r_stats = df_r_stats.set_index('epoch')
df_r_stats.to_csv('df_r_stats.csv')  
roberta_outputs_prob = np.array([np.array(xi) for xi in roberta_outputs_prob])
np.save('roberta_outputs_prob.npy',roberta_outputs_prob)

In [ ]:
import random

# This training code is based on the `run_glue.py` script here:
# a

# Set the seed value all over the place to make this reproducible.
seed_val = 100

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
xlnet_training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    xlnet_outputs_prob = []
    answers = []
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    xlnet_model.train()

    # For each batch of training data...
    for step, batch in enumerate(xlnet_train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(xlnet_train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)


        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        xlnet_model.zero_grad()        

        # Perform a forward pass (evaluate the roberta_model on this training batch).
        # The documentation for this `roberta_model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/roberta_model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # are given and what flags are set. For our usage here, it returns
        # the loss (because we provided labels) and the "logits"--the roberta_model
        # outputs prior to activation.


        outputs = xlnet_model(b_input_ids, 
                            #  token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.

        total_train_loss += outputs.loss.item()

        # Perform a backward pass to calculate the gradients.
        outputs.loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(xlnet_model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The roberta_optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        xlnet_optimizer.step()

        # Update the learning rate.
        xlnet_scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(xlnet_train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the roberta_model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    xlnet_model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in xlnet_validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():
       

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # Get the "logits" output by the roberta_model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            outputs = xlnet_model(b_input_ids, 
#                                    token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
        # Accumulate the validation loss.
        total_eval_loss += outputs.loss.item()
        answers.append(b_labels.tolist())

        for logit in outputs.logits:
          xlnet_outputs_prob.append(logit.tolist())

        # Move logits and labels to CPU
        outputs.logits = outputs.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(outputs.logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(xlnet_validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(xlnet_validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    xlnet_training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 5 ========
Training...
  Batch    40  of    214.    Elapsed: 0:00:22.
  Batch    80  of    214.    Elapsed: 0:00:44.
  Batch   120  of    214.    Elapsed: 0:01:06.
  Batch   160  of    214.    Elapsed: 0:01:27.
  Batch   200  of    214.    Elapsed: 0:01:47.

  Average training loss: 0.27
  Training epcoh took: 0:01:54

Running Validation...
  Accuracy: 0.96
  Validation Loss: 0.15
  Validation took: 0:00:09

======== Epoch 2 / 5 ========
Training...
  Batch    40  of    214.    Elapsed: 0:00:19.
  Batch    80  of    214.    Elapsed: 0:00:38.
  Batch   120  of    214.    Elapsed: 0:00:57.
  Batch   160  of    214.    Elapsed: 0:01:16.
  Batch   200  of    214.    Elapsed: 0:01:35.

  Average training loss: 0.10
  Training epcoh took: 0:01:42

Running Validation...
  Accuracy: 0.96
  Validation Loss: 0.10
  Validation took: 0:00:09

======== Epoch 3 / 5 ========
Training...
  Batch    40  of    214.    Elapsed: 0:00:19.
  Batch    80  of    214.    Elapsed: 0:00:38.
 

In [ ]:
df_x_stats = pd.DataFrame(data=xlnet_training_stats)
df_x_stats = df_x_stats.set_index('epoch')
df_x_stats.to_csv('df_x_stats.csv')  
xlnet_outputs_prob = np.array([np.array(xi) for xi in xlnet_outputs_prob])
np.save('xlnet_outputs_prob.npy',xlnet_outputs_prob)

In [ ]:
e_answers = []
for i in answers:
    for j in i:
        e_answers.append(j)

In [ ]:
np.save('anwers.npy',np.array(e_answers))

## Ensemble


In [ ]:
e_answers = []
for i in answers:
    for j in i:
        e_answers.append(j)

In [ ]:
def hard(logits):
  return np.argmax(logits, axis=1).flatten()

In [ ]:
def soft(logits):
  m = nn.Softmax(dim=1)
  output = m(torch.tensor(logits))
  return output

In [ ]:
soft_voting = torch.add(soft(xlnet_outputs_prob),soft(bert_outputs_prob))
soft_voting = torch.add(soft_voting, soft(roberta_outputs_prob))
soft_voting = torch.add(soft_voting, soft(ctbert_outputs_prob))
soft_voting = torch.add(soft_voting, soft(btbert_outputs_prob))
soft_labels = np.argmax(soft_voting, axis=1).flatten()

In [ ]:
hard_voting = hard(bert_outputs_prob) + hard(roberta_outputs_prob) + hard(xlnet_outputs_prob) + hard(ctbert_outputs_prob) + hard(btbert_outputs_prob)
hard_labels = []
for vote in hard_voting:
  if vote >= 2:
    hard_labels.append(1)
  else:
    hard_labels.append(0)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import matplotlib.pyplot as plt

In [ ]:
conf_matrix_soft = confusion_matrix(y_true=soft_labels, y_pred=e_answers)
conf_matrix_hard = confusion_matrix(y_true=hard_labels, y_pred=e_answers)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
ax.matshow(conf_matrix_soft, cmap=plt.cm.Oranges, alpha=0.3)
for i in range(conf_matrix_soft.shape[0]):
    for j in range(conf_matrix_soft.shape[1]):
        ax.text(x=j, y=i,s=conf_matrix_soft[i, j], va='center', ha='center', size='xx-large')
 
plt.xlabel('Predictions', fontsize=18)
plt.ylabel('Actuals', fontsize=18)
plt.title('Confusion Matrix Soft Voting', fontsize=18)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
ax.matshow(conf_matrix_hard, cmap=plt.cm.Oranges, alpha=0.3)
for i in range(conf_matrix_hard.shape[0]):
    for j in range(conf_matrix_hard.shape[1]):
        ax.text(x=j, y=i,s=conf_matrix_hard[i, j], va='center', ha='center', size='xx-large')
 
plt.xlabel('Predictions', fontsize=18)
plt.ylabel('Actuals', fontsize=18)
plt.title('Confusion Matrix Hard Voting', fontsize=18)
plt.show()

In [ ]:
print('Accuracy for soft voting: %.3f' % accuracy_score(soft_labels, e_answers))
print('Accuracy for hard voting: %.3f' % accuracy_score(hard_labels, e_answers))
print('F1 Score for soft voting: %.3f' % f1_score(soft_labels, e_answers))
print('F1 Score for hard voting: %.3f' % f1_score(hard_labels, e_answers))

In [ ]:
import pandas as pd

# Create a DataFrame from our training statistics.
df_b_stats = pd.DataFrame(data=bert_training_stats)
df_r_stats = pd.DataFrame(data=roberta_training_stats)
df_x_stats = pd.DataFrame(data=xlnet_training_stats)
df_c_stats = pd.DataFrame(data=ctbert_training_stats)

# Use the 'epoch' as the row index.
df_b_stats = df_b_stats.set_index('epoch')
df_r_stats = df_r_stats.set_index('epoch')
df_x_stats = df_x_stats.set_index('epoch')
df_c_stats = df_c_stats.set_index('epoch')

# A hack to force the column headers to wrap.
#df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

# Display the table.
df_b_stats

In [ ]:
df_r_stats

In [ ]:
df_x_stats

In [ ]:
df_c_stats